<a href="https://colab.research.google.com/github/vigneshwaran/MaLe/blob/master/Copy_of_imdb_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Trained first 1000 samples, got 52% accuracy \
Applied weight decay and weight initialization \
Trained over all samples with all words, got 60-63% accuracy

Movie review is overloaded with artist name, character name, movie names, some plots and stopwords. \
To select important words from samples, used stanford word embeddings that provided sentiment of each words,
got accuracy of over 70%.

In [0]:
import torch
import torch.nn as nn
import os
import re
import random
import math

In [0]:
# ls

In [3]:
# Dataset
!wget https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz 

--2020-05-06 06:17:24--  https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  60.9MB/s    in 1.3s    

2020-05-06 06:17:25 (60.9 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [0]:
!tar -xf aclImdb_v1.tar.gz

In [5]:
ls

aclImdb/  aclImdb_v1.tar.gz  sample_data/


In [0]:
path = os.getcwd()

In [0]:
path += '/aclImdb/'

In [0]:
# Path containing training data
filepath = ['/train/pos/', '/train/neg/']

In [0]:
# Each file name is given as fileindex_sentiment.txt

!ls aclImdb/train/pos/ | head -5

0_9.txt
10000_8.txt
10001_10.txt
10002_7.txt
10003_8.txt


In [9]:
!wget https://github.com/allenai/spv2/raw/master/model/glove.6B.100d.txt.gz

--2020-05-06 06:17:49--  https://github.com/allenai/spv2/raw/master/model/glove.6B.100d.txt.gz
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/allenai/spv2/master/model/glove.6B.100d.txt.gz [following]
--2020-05-06 06:17:49--  https://media.githubusercontent.com/media/allenai/spv2/master/model/glove.6B.100d.txt.gz
Resolving media.githubusercontent.com (media.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 134409071 (128M) [application/octet-stream]
Saving to: ‘glove.6B.100d.txt.gz’

glove.6B.100d.txt.g 100%[===================>] 128.18M   279MB/s    in 0.5s    

2020-05-06 06:17:52 (279 MB/s) - ‘glove.6B.100d.txt.gz’ saved [

In [0]:
!gunzip -c glove.6B.100d.txt.gz > glove.6B.100d.txt

In [0]:
ls -al

total 470260
drwxr-xr-x 1 root root      4096 May  5 08:26 ./
drwxr-xr-x 1 root root      4096 May  5 08:16 ../
drwxr-xr-x 1 root root      4096 Apr 29 16:14 .config/
-rw-r--r-- 1 root root 347116733 May  5 08:26 glove.6B.100d.txt
-rw-r--r-- 1 root root 134409071 May  5 08:22 glove.6B.100d.txt.gz
drwxr-xr-x 1 root root      4096 Apr  3 16:24 sample_data/


In [0]:
# # Pretrained word embedding, GLOVE
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip glove.6B.zip

--2020-05-04 07:26:10--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2020-05-04 07:26:10--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2020-05-04 07:26:10--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glove.6

In [0]:
# !head glove.6B.50d.txt

In [0]:
def read_glove_vecs(glove_file):
    """
    Returns word to index, index to words, word vector of words in form of dictionary
    """
    with open(glove_file, 'r',encoding='UTF-8') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = torch.FloatTensor(list(map(float,line[1:])))
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [0]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('glove.6B.100d.txt')

In [0]:
# Storing word embeds of review and its corresponding sentiment.  size=(N,2) 
# [word vectors, sentiment]
samples_we = []

In [0]:
""" Read each file in train directory and convert sentences to word vectors"""
for directory in filepath:
    for index, file in enumerate(os.listdir(path + directory)):
        with open(path + directory + file, 'r') as f:
            text = f.read().strip('\n').lower()
            words = re.findall('[a-z]+', text)  # selecting only alphabets from review
            weights = []   # word embeds
            for word in words:
              if word_to_index.get(word):
                weights.append(word_to_vec_map[word])
              else:
                continue
            weights = torch.stack(weights, dim=1) 
            weights = weights.view(100,1,-1)   # (input_size, batch_size, time_steps)
            match = re.search('_[0-9]+', file)
            Y = 0.1 * int(match.group()[1:])
            Y = torch.FloatTensor([Y])
            Y = Y.view(1,1)
            samples_we.append((weights, Y))   # (word embeds, sentiment)

In [15]:
samples_we[0][0].shape

torch.Size([100, 1, 236])

In [0]:
random.shuffle(samples_we)

In [0]:
def sigmoid_prime(z):
	  return sigmoid(z)*(1- sigmoid(z))

In [0]:
def sigmoid(x):
    return 1 / (1 + torch.exp(-x))

In [0]:
class RNN():
    def __init__(self, input_size, hidden_size, output_size, batch_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.batch_size = batch_size
#         self.ts = ts  # time step
        # self.parameters = {'Waa' : torch.randn((hidden_size, hidden_size))*math.sqrt(1/(hidden_size)),
        #                    'Wax' : torch.randn((hidden_size, input_size))*math.sqrt(1/(hidden_size)),
        #                    'Wya' : torch.randn((output_size, hidden_size))*math.sqrt(1/(output_size)),
        #                    'ba'  : torch.zeros((hidden_size, 1)),
        #                    'by'  : torch.zeros((output_size, 1))
        #                   }
        self.parameters = {'Waa' : torch.randn((hidden_size, hidden_size))*0.01,
                           'Wax' : torch.randn((hidden_size, input_size))*0.01 ,
                           'Wya' : torch.randn((output_size, hidden_size))*0.01,
                           'ba'  : torch.zeros((hidden_size, 1)),
                           'by'  : torch.zeros((output_size, 1))
                          }
        for i in self.parameters.keys():
            self.parameters[i] = self.parameters[i].to(device)
         
    def forward(self, X):
        self.a_list = [torch.zeros(self.hidden_size, X.shape[1]).to(device)]   # a_list stores output of every activation function
        self.a_out = [torch.zeros(self.output_size, X.shape[1]).to(device)]    # a_out stores output of final activation function
        a_prev = torch.randn(self.hidden_size, X.shape[1]).to(device)
        self.X = X.to(device)
        self.ts = X.shape[-1]
        for t in range(self.ts):
            xt = X[:, :, t]
            a_prev = torch.tanh(torch.matmul(self.parameters['Waa'], a_prev) + 
                                torch.matmul(self.parameters['Wax'], xt) + self.parameters['ba'])
            self.a_list.append(a_prev)
        a_Out = torch.matmul(self.parameters['Wya'], a_prev) + self.parameters['by']
        y_pred = sigmoid(a_Out)
        self.a_out.append(a_Out)
        self.Y = y_pred
        return self.Y
    
    def getY(self):
        return self.Y
    
    def getHidden(self):
        return self.hidden_size, self.batch_size
    
    def normalize(self):
        # Normalize into uniform distribution
        for i, j in self.parameters.items():
            self.parameters[i] = (j - j.mean(dim = -1, keepdim = True))/j.std(dim = -1, keepdim = True)
    def normalise(self):
        for i, j in self.parameters.items():
            self.parameters[i] = j.normal_()
    def normalize_tensor(self, l):
        return (l - l.mean(dim = -1, keepdim = True))/l.std(dim = -1, keepdim = True)

    def clip(self, l):
      # To avoid exploding gradient descent, its value is clipped to a range
      clamped = []
      for i in range(len(l)):
        clamped.append(torch.clamp(l[i], min = -0.5, max = 0.5))
      return clamped
            
                                
    def backward(self, y, lr=0.001, lmbda = 1.0e-2):
        bs = y.shape[-1]
        da = self.a_list[0].to(device)
        dx = torch.zeros(self.X.shape).to(device)
        dWax = torch.zeros((self.parameters['Wax'].shape)).to(device)
        dWaa = torch.zeros((self.parameters['Waa'].shape)).to(device)
        dba = torch.zeros((self.parameters['ba'].shape)).to(device)
        dWya = torch.zeros((self.parameters['Wya'].shape)).to(device)
        dby = torch.zeros((self.parameters['by'].shape)).to(device)
        
        # Cost function
        losses = torch.sum(-1.0 * (y.mm(torch.log(self.Y.t()))))

        # Calculating gradients
        cost = (self.Y - y)
        dWya += torch.mm(cost, self.a_out[-1].t())
        dby += torch.sum(cost, axis = 1, keepdims=True)
        da += torch.mm(self.parameters['Wya'].t(), cost)

        ts = self.ts
        if ts < 1:
          return 0
        for t in reversed(range(ts)):
            xt = self.X[:, :, t]
            dz = da * (1 - (self.a_list[t])**2).to(device)
            dxt = torch.mm(self.parameters['Wax'].t(), dz)
            dx[:, :, t] = dxt
            dWax += torch.mm(dz, xt.t())
            da = torch.mm(self.parameters['Waa'].t(), dz)
            dWaa += torch.mm(dz, self.a_list[t-1].t())
            dba += torch.sum(dz, axis = 1, keepdims=True)
            

        (dWax, dWya, dWaa, dba, dby) = self.clip([dWax/ts, dWya/ts, dWaa/ts, dba/ts, dby/ts])
        # self.parameters['Wax'] = (1-lr*(lmbda/bs*ts))*self.parameters['Wax']-lr*dWax
        # self.parameters['Wya'] = (1-lr*(lmbda/bs*ts))*self.parameters['Wya']-lr*dWya
        # self.parameters['Waa'] = (1-lr*(lmbda/bs*ts))*self.parameters['Waa']-lr*dWaa
        # self.parameters['ba']  -= lr*dba/self.ts
        # self.parameters['by']  -= lr*dby/self.ts
        self.parameters['Wax'] -= lr*dWax
        self.parameters['Wya'] -= lr*dWya
        self.parameters['Waa'] -= lr*dWaa
        self.parameters['ba']  -= lr*dba
        self.parameters['by']  -= lr*dby
        return losses

In [20]:
# Dimension of word vector is 50, from GLOVE
# Treated each review as a batch
sentiment_glove = RNN(input_size = 100, hidden_size = 256, output_size = 1, batch_size = 1)

NameError: ignored

In [0]:
# sentiment_glove.parameters

In [0]:
"""Training LSTM on data containing variable batch size each having same number of words, used GLOVE as word vector"""
for epoch in range(20):
  total_cost = 0
  acc = 0
  sz = 0
  for i in batches:
    sz += i[1].shape[-1]
    pred = sentiment_glove.forward(i[0].to(device))
    acc += torch.sum(pred.round() == i[1].to(device).round())
    total_cost += sentiment_glove.backward(i[1].to(device), lr=0.001)
  print(epoch, total_cost, acc.item()/sz)

KeyboardInterrupt: ignored

In [0]:
# Training over first 1000 samples, used Glove as word vector
for epoch in range(100):
  total_cost = 0
  acc = 0
  for i,j in samples_we[:1000]:
    pred = sentiment_glove.forward(i)
    acc += (pred.round() == j.round())
    total_cost += sentiment_glove.backward(j, lr=1e-3)
  print(epoch, total_cost, acc.item()/10)

0 tensor(381.8438) 52.4
1 tensor(381.6916) 51.3
2 tensor(381.5406) 50.9
3 tensor(381.3897) 51.3
4 tensor(381.2391) 52.4
5 tensor(381.0887) 50.8
6 tensor(380.9391) 49.4
7 tensor(380.7888) 49.7
8 tensor(380.6401) 49.0
9 tensor(380.4915) 48.6


In [0]:
""" Collecting train dataset"""
test_filepath = ['/test/pos/', 'test/neg/']
test_we = []

In [0]:
for directory in test_filepath:
    for index, file in enumerate(os.listdir(path + directory)):
        with open(path + directory + file, 'r') as f:
            text = f.read().strip('\n').lower()
            words = re.findall('[a-z]+', text)
            weights = []
            for word in words:
              if word_to_index.get(word):
                weights.append(word_to_vec_map[word])
              else:
                weights.append(word_to_vec_map['unk'])
            weights = torch.stack(weights, dim=1) 
            weights = weights.view(50,1,-1)
            match = re.search('_[0-9]+', file)
            Y = 0.1 * int(match.group()[1:])
            Y = torch.FloatTensor([Y])
            Y = Y.view(1,1)
            test_we.append((weights, Y))

In [0]:
# Evaluation
total_cost = 0
acc = 0
for i,j in test_we:
  pred = sentiment_glove.forward(i)
  acc += (pred.round() == j.round())
  # total_cost += sentiment_glove.backward(j, lr=0.001)
print(acc.item()/len(test_we)*100)

49.94


Batch size = 1, does not produced expected result. \
As sentences have variable length, could not group together into same batch. \
Sentences of same length is onsidered as same batch


In [0]:
# Sorted word embeddings list based on number of words in sample.
# Samples containing same number of words are choosen as same batch
batches = []
samples_we.sort(key= lambda s: s[0].size(-1)) 
x = [samples_we[0][0]]
y = [samples_we[0][1]]
for i in range(1, len(samples_we)):
  if samples_we[i][0].size(-1) == samples_we[i-1][0].size(-1):
    x.append(samples_we[i][0])
    y.append(samples_we[i][1])
  else:
    x = torch.stack(x, dim = 2)
    x = x.view(100, x.shape[2], -1)
    y = torch.stack(y, dim = 2)
    y = y.view(1, y.shape[2])
    batches.append((x,y))
    x = [samples_we[i][0]]
    y = [samples_we[i][1]]
x = torch.stack(x, dim = 2)
x = x.view(100, x.shape[2], -1)
y = torch.stack(y, dim = 2)
y = y.view(1, y.shape[2])
batches.append((x,y))

In [26]:
batches[-1][0].shape  # (dimension of word vector, batch size, number of words in batch)

torch.Size([100, 1, 2504])

In [27]:
batches[1][0].shape

torch.Size([100, 2, 11])

In [0]:
len(batches)

In [0]:
# Dimension of word vector is 50, from GLOVE
sentiment_glove = RNN(input_size = 100, hidden_size = 256, output_size = 1, batch_size = 1)

In [0]:
"""Training on data containing variable batch size each having same number of words, used Glove as word vector"""
for epoch in range(50):
  total_cost = 0
  acc = 0
  sz = 0
  for i in batches:
    sz += i[1].shape[-1]
    pred = sentiment_glove.forward(i[0])
    acc += torch.sum(pred.round() == i[1].round())
    total_cost += sentiment_glove.backward(i[1], lr=0.001)
  print(epoch, total_cost, acc.item()/sz)

KeyboardInterrupt: ignored

Each review is too long, RNN does not perform well. \
Tried to implement LSTM,

In [0]:
torch.randn??

In [24]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print (device)

cuda:0


In [0]:
class lstmRNN(nn.Module): # LSTM
    def __init__(self, input_size, hidden_size, output_size, batch_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.batch_size = batch_size
#         self.ts = ts  # time step
        self.parameters = {'Wf' : torch.randn((hidden_size, input_size + hidden_size)),
                           'Wi' : torch.randn((hidden_size, input_size + hidden_size)),
                           'Wo' : torch.randn((hidden_size, input_size + hidden_size)),
                           'Wc' : torch.randn((hidden_size, input_size + hidden_size)),
                           'Wy' : torch.randn((output_size, hidden_size)),
                           'bf' : torch.randn((hidden_size, 1)),
                           'bi' : torch.randn((hidden_size, 1)),
                           'bo' : torch.randn((hidden_size, 1)),
                           'bc' : torch.randn((hidden_size, 1)),
                           'by' : torch.randn((output_size,1))
                          }
        for i in self.parameters.keys():
            self.parameters[i] = nn.init.xavier_uniform_(self.parameters[i]).to(device)

         
    def forward(self, X):
        self.a = torch.zeros(self.hidden_size, X.shape[1], X.shape[-1]).to(device)
        self.c = torch.zeros(self.hidden_size, X.shape[1], X.shape[-1]).to(device)
        self.y = torch.zeros(self.output_size, X.shape[1], X.shape[-1]).to(device)
        a_prev = torch.randn(self.hidden_size, X.shape[1]).to(device)
        c_prev = torch.zeros(self.hidden_size, X.shape[1]).to(device)
        self.X = X.to(device)
        self.ts = X.shape[-1]
        self.ft = []
        self.it = []
        self.ot = []
        self.cct = []
        for t in range(self.ts):
            xt = X[:, :, t]
            concat = torch.cat((a_prev, xt), dim = 0)
            ft = sigmoid (torch.mm(self.parameters['Wf'], concat) + self.parameters['bf'])
            it = sigmoid (torch.mm(self.parameters['Wi'], concat) + self.parameters['bi'])
            cct = torch.tanh(torch.mm(self.parameters['Wc'], concat) + self.parameters['bc'])
            c_prev = torch.mul(ft, c_prev) + torch.mul(it, cct)
            ot = sigmoid (torch.mm(self.parameters['Wo'], concat) + self.parameters['bo'])
            a_prev = torch.mul(ot, torch.tanh(c_prev))
            self.a[:,:,t] = a_prev
            self.c[:,:,t] = c_prev
            self.ft.append(ft)
            self.it.append(it)
            self.ot.append(ot)
            self.cct.append(cct)
        self.a_out = torch.matmul(self.parameters['Wy'], a_prev) + self.parameters['by'].to(device)
        y_pred = sigmoid(self.a_out)
        self.Y = y_pred
        return self.Y
    
    
    def normalize(self):
        # Normalize into standard distribution
        for i, j in self.parameters.items():
          if j.shape[1] > 1:
            self.parameters[i] = (j - j.mean(dim = 1, keepdim = True))/j.std(dim = 1, keepdim = True)

    def clip(self, grad):
        for i in grad.keys():
          grad[i] = torch.clamp(grad[i], -0.5, 0.5, out = grad[i])
        return grad

    def update_parameters(self, gradients, ts, lr = 0.0001):
        for i, j in self.parameters.items():
            self.parameters[i] -= lr * gradients['d' + i]


    def squared_sum(self):
      total = 0
      for i in self.parameters.keys():
        total += torch.sum((self.parameters[i])**2)
      return total
                                    
    def backward1(self, y, lr=0.0001, lmbda = 1.0e-2):
        
        grad = {}
        grad['dWf'] = torch.zeros((self.parameters['Wf'].shape)).to(device)
        grad['dWi'] = torch.zeros((self.parameters['Wi'].shape)).to(device)
        grad['dWo'] = torch.zeros((self.parameters['Wo'].shape)).to(device)
        grad['dWc'] = torch.zeros((self.parameters['Wc'].shape)).to(device)
        grad['dWy'] = torch.zeros((self.parameters['Wy'].shape)).to(device)
        grad['dbf'] = torch.zeros((self.parameters['bf'].shape)).to(device)
        grad['dbi'] = torch.zeros((self.parameters['bi'].shape)).to(device)
        grad['dbo'] = torch.zeros((self.parameters['bo'].shape)).to(device)
        grad['dbc'] = torch.zeros((self.parameters['bc'].shape)).to(device)
        grad['dby'] = torch.zeros((self.parameters['by'].shape)).to(device)
        
        
        
        bs = y.shape[-1]
        ts = self.ts
        da = torch.zeros(self.hidden_size, bs).to(device)
        dc = torch.zeros(self.hidden_size, bs).to(device) 
        cost = (self.Y - y) 
        cost += (1/bs*lmbda/2)*squared_sum()
        
        grad['dWy'] += (lmbda/bs)*torch.mm(cost, self.a_out.t())
        grad['dby'] += (1/bs)*torch.sum(cost, axis = 1, keepdims=True)
        
        da += torch.mm(self.parameters['Wy'].t(), cost)
        losses = torch.sum(-1.0 * (y * (torch.log(self.Y))))
        ot = self.ot
        it = self.it
        cct = self.cct
        ft = self.ft
        a = self.a
        c = self.c
        if ts < 1:
            return 0
        for t in reversed(range(ts)):
            xt = self.X[:, :, t]
            
#             print(da.shape, c[:,:,t].shape, ot[t].shape, ot[t-1].shape)
            dot = da * (torch.tanh(c[:,:,t])) * ot[t] * (1 - ot[t])
            temp = ot[t] * (1 - (torch.tanh(c[:,:,t])**2))
            dcct = dc * it[t] + temp * it[t] * da * (1 - (cct[t]**2))
            dit = dc * cct[t] + temp * cct[t] * da * (it[t] * (1 - it[t]))
            dft = dc * c[:,:,t-1] + temp * c[:,:,t-1] * da * (ft[t] * (1 - ft[t]))
            concat = torch.cat((a[:, :, t-1], xt), dim = 0)

            grad['dWf'] += (lmbda/(bs*ts))*torch.mm(dft, concat.t())
            grad['dWi'] += (lmbda/(bs*ts))*torch.mm(dit, concat.t())
            grad['dWc'] += (lmbda/(bs*ts))*torch.mm(dcct, concat.t())
            grad['dWo'] += (lmbda/(bs*ts))*torch.mm(dot, concat.t())
            grad['dbf'] += (1/(bs*ts))*torch.sum(dft, axis = 1, keepdims=True)
            grad['dbi'] += (1/(bs*ts))*torch.sum(dit, axis = 1, keepdims=True)
            grad['dbc'] += (1/(bs*ts))*torch.sum(dcct, axis = 1, keepdims=True)
            grad['dbo'] += (1/(bs*ts))*torch.sum(dot, axis = 1, keepdims=True)

            da = torch.mm((self.parameters['Wf'][:, : self.hidden_size]).t(), dft) + torch.mm((self.parameters['Wi'][:, : self.hidden_size]).t(), dit) + torch.mm((self.parameters['Wc'][:, : self.hidden_size]).t(), dcct) + torch.mm((self.parameters['Wo'][:, : self.hidden_size]).t(), dot)
            dc = dc * ft[t] + ot[t] * (1 - (torch.tanh(c[:,:,t])**2)) * da * ft[t]

            
        self.grad = self.clip(grad)
        self.update_parameters(self.grad, ts, lr)
        return losses

In [0]:
class lstmRNN(nn.Module): # LSTM
    def __init__(self, input_size, hidden_size, output_size, batch_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.batch_size = batch_size
#         self.ts = ts  # time step
        self.parameters = {'Wf' : torch.randn((hidden_size, input_size + hidden_size)),
                           'Wi' : torch.randn((hidden_size, input_size + hidden_size)),
                           'Wo' : torch.randn((hidden_size, input_size + hidden_size)),
                           'Wc' : torch.randn((hidden_size, input_size + hidden_size)),
                           'Wy' : torch.randn((output_size, hidden_size)),
                           'bf' : torch.randn((hidden_size, 1)),
                           'bi' : torch.randn((hidden_size, 1)),
                           'bo' : torch.randn((hidden_size, 1)),
                           'bc' : torch.randn((hidden_size, 1)),
                           'by' : torch.randn((output_size,1))
                          }
        for i in self.parameters.keys():
            self.parameters[i] = nn.init.xavier_uniform_(self.parameters[i]).to(device)

         
    def forward(self, X):
        self.a = torch.zeros(self.hidden_size, X.shape[1], X.shape[-1]).to(device)
        self.c = torch.zeros(self.hidden_size, X.shape[1], X.shape[-1]).to(device)
        self.y = torch.zeros(self.output_size, X.shape[1], X.shape[-1]).to(device)
        a_prev = torch.randn(self.hidden_size, X.shape[1]).to(device)
        c_prev = torch.zeros(self.hidden_size, X.shape[1]).to(device)
        self.X = X.to(device)
        self.ts = X.shape[-1]
        self.ft = []
        self.it = []
        self.ot = []
        self.cct = []
        for t in range(self.ts):
            xt = X[:, :, t]
            concat = torch.cat((a_prev, xt), dim = 0)
            ft = sigmoid (torch.mm(self.parameters['Wf'], concat) + self.parameters['bf'])
            it = sigmoid (torch.mm(self.parameters['Wi'], concat) + self.parameters['bi'])
            cct = torch.tanh(torch.mm(self.parameters['Wc'], concat) + self.parameters['bc'])
            c_prev = torch.mul(ft, c_prev) + torch.mul(it, cct)
            ot = sigmoid (torch.mm(self.parameters['Wo'], concat) + self.parameters['bo'])
            a_prev = torch.mul(ot, torch.tanh(c_prev))
            self.a[:,:,t] = a_prev
            self.c[:,:,t] = c_prev
            self.ft.append(ft)
            self.it.append(it)
            self.ot.append(ot)
            self.cct.append(cct)
        self.a_out = torch.matmul(self.parameters['Wy'], a_prev) + self.parameters['by'].to(device)
        y_pred = sigmoid(self.a_out)
        self.Y = y_pred
        return self.Y
    
    
    def normalize(self):
        # Normalize into standard distribution
        for i, j in self.parameters.items():
          if j.shape[1] > 1:
            self.parameters[i] = (j - j.mean(dim = 1, keepdim = True))/j.std(dim = 1, keepdim = True)

    def clip(self, grad):
        for i in grad.keys():
          grad[i] = torch.clamp(grad[i], -0.5, 0.5, out = grad[i])
        return grad

    def update_parameters(self, gradients, ts, lr = 0.0001):
        for i, j in self.parameters.items():
            self.parameters[i] -= lr * gradients['d' + i]
                                    
    def backward1(self, y, lr=0.0001, lmbda = 1.0e-2):
        
        grad = {}
        grad['dWf'] = torch.zeros((self.parameters['Wf'].shape)).to(device)
        grad['dWi'] = torch.zeros((self.parameters['Wi'].shape)).to(device)
        grad['dWo'] = torch.zeros((self.parameters['Wo'].shape)).to(device)
        grad['dWc'] = torch.zeros((self.parameters['Wc'].shape)).to(device)
        grad['dWy'] = torch.zeros((self.parameters['Wy'].shape)).to(device)
        grad['dbf'] = torch.zeros((self.parameters['bf'].shape)).to(device)
        grad['dbi'] = torch.zeros((self.parameters['bi'].shape)).to(device)
        grad['dbo'] = torch.zeros((self.parameters['bo'].shape)).to(device)
        grad['dbc'] = torch.zeros((self.parameters['bc'].shape)).to(device)
        grad['dby'] = torch.zeros((self.parameters['by'].shape)).to(device)
        
        
        
        bs = y.shape[-1]
        ts = self.ts
        da = torch.zeros(self.hidden_size, bs).to(device)
        dc = torch.zeros(self.hidden_size, bs).to(device) 
        cost = (self.Y - y)
        
        grad['dWy'] += torch.mm(cost, self.a_out.t())
        grad['dby'] += torch.sum(cost, axis = 1, keepdims=True)
        
        da += torch.mm(self.parameters['Wy'].t(), cost)
        losses = torch.sum(-1.0 * (y * (torch.log(self.Y))))
        ot = self.ot
        it = self.it
        cct = self.cct
        ft = self.ft
        a = self.a
        c = self.c
        if ts < 1:
            return 0
        for t in reversed(range(ts)):
            xt = self.X[:, :, t]
            
#             print(da.shape, c[:,:,t].shape, ot[t].shape, ot[t-1].shape)
            dot = da * (torch.tanh(c[:,:,t])) * ot[t] * (1 - ot[t])
            temp = ot[t] * (1 - (torch.tanh(c[:,:,t])**2))
            dcct = dc * it[t] + temp * it[t] * da * (1 - (cct[t]**2))
            dit = dc * cct[t] + temp * cct[t] * da * (it[t] * (1 - it[t]))
            dft = dc * c[:,:,t-1] + temp * c[:,:,t-1] * da * (ft[t] * (1 - ft[t]))
            concat = torch.cat((a[:, :, t-1], xt), dim = 0)

            grad['dWf'] += torch.mm(dft, concat.t())
            grad['dWi'] += torch.mm(dit, concat.t())
            grad['dWc'] += torch.mm(dcct, concat.t())
            grad['dWo'] += torch.mm(dot, concat.t())
            grad['dbf'] += torch.sum(dft, axis = 1, keepdims=True)
            grad['dbi'] += torch.sum(dit, axis = 1, keepdims=True)
            grad['dbc'] += torch.sum(dcct, axis = 1, keepdims=True)
            grad['dbo'] += torch.sum(dot, axis = 1, keepdims=True)

            da = torch.mm((self.parameters['Wf'][:, : self.hidden_size]).t(), dft) + torch.mm((self.parameters['Wi'][:, : self.hidden_size]).t(), dit) + torch.mm((self.parameters['Wc'][:, : self.hidden_size]).t(), dcct) + torch.mm((self.parameters['Wo'][:, : self.hidden_size]).t(), dot)
            dc = dc * ft[t] + ot[t] * (1 - (torch.tanh(c[:,:,t])**2)) * da * ft[t]

            
        self.grad = self.clip(grad)
        self.update_parameters(self.grad, ts, lr)
        return losses

In [0]:
# Dimension of word vector is 50, from GLOVE
sentiment_glove = lstmRNN(input_size = 100, hidden_size = 256, output_size = 1, batch_size = 1)

In [0]:
# sentiment_glove.to(device)

In [84]:
"""Training LSTM on data containing variable batch size each having same number of words, used GLOVE as word vector"""
for epoch in range(100):
  total_cost = 0
  acc = 0
  sz = 0
  for i in batches[:100]:
    sz += i[1].shape[-1]
    pred = sentiment_glove.forward(i[0].to(device))
    acc += torch.sum(pred.round() == i[1].to(device).round())
    total_cost += sentiment_glove.backward1(i[1].to(device), lr=0.001)
  print(epoch, total_cost.item(), acc.item()/sz)

0 tensor(1180.6357, device='cuda:0') 0.5484318623369414
1 tensor(1131.7408, device='cuda:0') 0.5481543158479045
2 tensor(1124.3861, device='cuda:0') 0.549819594782126
3 tensor(1118.4504, device='cuda:0') 0.5525950596724951
4 tensor(1112.8575, device='cuda:0') 0.5575908964751596
5 tensor(1107.3179, device='cuda:0') 0.5678601165695254
6 tensor(1101.6173, device='cuda:0') 0.5767416042187067
7 tensor(1095.5621, device='cuda:0') 0.5872883708021094
8 tensor(1088.9807, device='cuda:0') 0.601165695253955
9 tensor(1081.8254, device='cuda:0') 0.6161532056619484
10 tensor(1074.1207, device='cuda:0') 0.6330835414932001
11 tensor(1066.0494, device='cuda:0') 0.6419650291423813
12 tensor(1057.6708, device='cuda:0') 0.6563974465723008
13 tensor(1048.8762, device='cuda:0') 0.6666666666666666
14 tensor(1039.1743, device='cuda:0') 0.6777685262281432
15 tensor(1022.9374, device='cuda:0') 0.6783236192062171
16 tensor(1031.8245, device='cuda:0') 0.6894254787676936
17 tensor(1025.8173, device='cuda:0') 0.700

In [0]:
# Dimension of word vector is 50, from GLOVE
sentiment_glove = lstmRNN(input_size = 100, hidden_size = 256, output_size = 1, batch_size = 1)

In [0]:
"""Training LSTM on data containing variable batch size each having same number of words, used GLOVE as word vector"""
for epoch in range(50):
  total_cost = 0
  acc = 0
  sz = 0
  for i in batches:
    sz += i[1].shape[-1]
    pred = sentiment_glove.forward(i[0].to(device))
    acc += torch.sum(pred.round() == i[1].to(device).round())
    total_cost += sentiment_glove.backward1(i[1].to(device), lr=0.001)
  print(epoch, total_cost, acc.item()/sz)

0 tensor(8658.1494, device='cuda:0') 0.52204
1 tensor(8198.2021, device='cuda:0') 0.56268
2 tensor(7960.6187, device='cuda:0') 0.614
3 tensor(7822.8477, device='cuda:0') 0.64908
4 tensor(7739.2041, device='cuda:0') 0.6652
5 tensor(7609.8638, device='cuda:0') 0.65632
6 tensor(7276.5137, device='cuda:0') 0.66344


In the dataset, the sentiment of each word is given \
Sentiment of words are choosen as word vector of dimension 1.

In [0]:
!head aclImdb/imdb.vocab

In [0]:
!head aclImdb/imdbEr.txt

In [0]:
word_idx = {} # storing index corresponding to words

In [0]:
# contains all distinct words
with open(path + '/imdb.vocab', 'r') as f:  
    out = f.readlines()
    for i, word in enumerate(out):
        word_idx[word.strip('\n')] = i

In [0]:
idx_sent = {}

In [0]:
# file contains sentiment of each corresponding word to vocab
with open(path + '/imdbEr.txt', 'r') as f:   
    out = f.readlines()
    for i, word in enumerate(out):
        idx_sent[i] = float(word.strip('\n'))

In [0]:
# Storing word embeds of review and its corresponding sentiment.  size=(N,2) 
samples = [] 

In [0]:
for directory in filepath:
    for index, file in enumerate(os.listdir(path + directory)):
        with open(path + directory + file, 'r') as f:
            text = f.read().strip('\n').lower()
            words = re.findall('[a-z]+', text)  # selecting only alphabets from review
            weights = []  # word embeds 
            for word in words:
              if word_idx.get(word):
                weights.append(idx_sent[word_idx[word]])
              else:
                weights.append(0.0)
            weights = torch.FloatTensor(weights) 
            weights = weights.view(1,1,-1) # (input_size, batch_size, time_steps)
            match = re.search('_[0-9]+', file)
            Y = 0.1 * int(match.group()[1:])
            Y = torch.FloatTensor([Y])
            Y = Y.view(1,1)
            samples.append((weights, Y)) # (word embeds, sentiment)

In [0]:
class RNN():
    def __init__(self, input_size, hidden_size, output_size, batch_size):
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.batch_size = batch_size
#         self.ts = ts  # time step
        # self.parameters = {'Waa' : torch.randn((hidden_size, hidden_size))*math.sqrt(1/(hidden_size)),
        #                    'Wax' : torch.randn((hidden_size, input_size))*math.sqrt(1/(hidden_size)),
        #                    'Wya' : torch.randn((output_size, hidden_size))*math.sqrt(1/(output_size)),
        #                    'ba'  : torch.zeros((hidden_size, 1)),
        #                    'by'  : torch.zeros((output_size, 1))
        #                   }
        self.parameters = {'Waa' : torch.randn((hidden_size, hidden_size))*0.01,
                           'Wax' : torch.randn((hidden_size, input_size))*0.01 ,
                           'Wya' : torch.randn((output_size, hidden_size))*0.01,
                           'ba'  : torch.zeros((hidden_size, 1)),
                           'by'  : torch.zeros((output_size, 1))
                          }
         
    def forward(self, X):
        self.a_list = [torch.zeros(self.hidden_size, X.shape[1])]   # a_list stores output of every activation function
        self.a_out = [torch.zeros(self.output_size, X.shape[1])]    # a_out stores output of final activation function
        a_prev = torch.randn(self.hidden_size, X.shape[1])
        self.X = X
        self.ts = X.shape[-1]
        for t in range(self.ts):
            xt = X[:, :, t]
            a_prev = torch.tanh(torch.matmul(self.parameters['Waa'], a_prev) + 
                                torch.matmul(self.parameters['Wax'], xt) + self.parameters['ba'])
            self.a_list.append(a_prev)
        a_Out = torch.matmul(self.parameters['Wya'], a_prev) + self.parameters['by']
        y_pred = sigmoid(a_Out)
        self.a_out.append(a_Out)
        self.Y = y_pred
        return self.Y
    
    
    def normalize(self):
        # Normalize into uniform distribution
        for i, j in self.parameters.items():
            self.parameters[i] = (j - j.mean())/j.std()
            

    def clip(self, l):
      # To avoid exploding gradient descent, its value is clipped to a range
      clamped = []
      for i in range(len(l)):
        clamped.append(torch.clamp(l[i], min = -0.5, max = 0.5))
      return clamped
            
                                
    def backward(self, y, lr=0.001, lmbda = 1.0e-2):
        bs = y.shape[-1]
        da = self.a_list[0]
        dx = torch.zeros(self.X.shape)
        dWax = torch.zeros((self.parameters['Wax'].shape))
        dWaa = torch.zeros((self.parameters['Waa'].shape))
        dba = torch.zeros((self.parameters['ba'].shape))
        dWya = torch.zeros((self.parameters['Wya'].shape))
        dby = torch.zeros((self.parameters['by'].shape))
        
        # Cost function
        losses = torch.sum(-1.0 * (y.mm(torch.log(self.Y.t()))))

        # Calculating gradients
        cost = (self.Y - y)
        dWya += torch.mm(cost, self.a_out[-1].t())
        dby += torch.sum(cost, axis = 1, keepdims=True)
        da += torch.mm(self.parameters['Wya'].t(), cost)

        ts = self.ts
        if ts < 1:
          return 0
        for t in reversed(range(0, ts)):
            xt = self.X[:, :, t]
            dz = da * (1 - (self.a_list[t])**2)
            dxt = torch.mm(self.parameters['Wax'].t(), dz)
            dx[:, :, t] = dxt
            dWax += torch.mm(dz, xt.t())
            da = torch.mm(self.parameters['Waa'].t(), dz)
            dWaa += torch.mm(dz, self.a_list[t-1].t())
            dba += torch.sum(dz, axis = 1, keepdims=True)
            

        (dWax, dWya, dWaa, dba, dby) = self.clip([dWax/ts, dWya/ts, dWaa/ts, dba/ts, dby/ts])
        # self.parameters['Wax'] = (1-lr*(lmbda/bs*ts))*self.parameters['Wax']-lr*dWax
        # self.parameters['Wya'] = (1-lr*(lmbda/bs*ts))*self.parameters['Wya']-lr*dWya
        # self.parameters['Waa'] = (1-lr*(lmbda/bs*ts))*self.parameters['Waa']-lr*dWaa
        # self.parameters['ba']  -= lr*dba/self.ts
        # self.parameters['by']  -= lr*dby/self.ts
        self.parameters['Wax'] -= lr*dWax
        self.parameters['Wya'] -= lr*dWya
        self.parameters['Waa'] -= lr*dWaa
        self.parameters['ba']  -= lr*dba
        self.parameters['by']  -= lr*dby
        return losses

In [0]:
# Dimension of word vector is one, word sentiment
sentiment = RNN(input_size = 1, hidden_size = 3, output_size = 1, batch_size = 1)

In [0]:
"""Training on data containing batch size = 1, word sentiments are choosen as word vector"""
for epoch in range(10):
  total_cost = 0
  acc = 0
  for i,j in samples:
    pred = sentiment.forward(i)
    acc += (pred.round() == j.round())
    total_cost += sentiment.backward(j, lr=0.001)
    # if (count % 1000 == 0):
    #   print(acc)
    #   acc = 0
    # count += 1
  print(epoch, total_cost, (acc/len(samples))*10)

KeyboardInterrupt: ignored

Used sentiment of each word, but didn't find useful results. \
Selected only important words by applying threshold value.   \

In [0]:
# Only words having sentiment greater than 1.0 or less than -1.0

samples = []
for directory in filepath:
    for index, file in enumerate(os.listdir(path + directory)):
        with open(path + directory + file, 'r') as f:
            text = f.read().strip('\n').lower()
            words = re.findall('[a-z]+', text)
            weights = []
            for word in words:
              if word_idx.get(word) and abs(idx_sent[word_idx[word]]) >= 1.0: # threshold to priotize only important words
                weights.append(idx_sent[word_idx[word]])
              else:
                continue
            weights = torch.FloatTensor(weights) 
            weights = weights.view(1,1,-1) # (input_size, batch_size, time_steps)
            match = re.search('_[0-9]+', file)
            Y = 0.1 * int(match.group()[1:])
            Y = torch.FloatTensor([Y])
            Y = Y.view(1,1)
            samples.append((weights, Y))

In [0]:
# samples_sorted based on number of words
samples.sort(key= lambda s: s[0].size(-1))

In [0]:
# Clustering batches of same length
batches_s = []
x = [samples[0][0]]
y = [samples[0][1]]
for i in range(1, len(samples)):
  if samples[i][0].size(-1) == samples[i-1][0].size(-1):
    x.append(samples[i][0])
    y.append(samples[i][1])
  else:
    x = torch.stack(x, dim = 2)
    x = x.view(1, x.shape[2], -1)
    y = torch.stack(y, dim = 2)
    y = y.view(1, y.shape[2])
    batches_s.append((x,y))
    x = [samples[i][0]]
    y = [samples[i][1]]
x = torch.stack(x, dim = 2)
x = x.view(1, x.shape[2], -1)
y = torch.stack(y, dim = 2)
y = y.view(1, y.shape[2])
batches_s.append((x,y))

In [0]:
# Dimension of word vector is 1, word sentiment
sentiment = RNN(input_size = 1, hidden_size = 64, output_size = 1)

TypeError: ignored

In [0]:
"""Training on data containing variable batch size each having same number of words, word sentiments are choosen as word vector with threshold"""
for epoch in range(40):
  total_cost = 0
  acc = 0
  sz = 0
  for i in batches_s:
    sz += i[1].shape[-1]
    pred = sentiment.forward(i[0])
    acc += torch.sum(pred.round() == i[1].round())
    total_cost += sentiment.backward(i[1], lr=0.01)

  print(epoch, total_cost, acc.item()/sz)

0 tensor(8792.1240) 0.55592
1 tensor(8005.7119) 0.72696
2 tensor(7303.8335) 0.74296
3 tensor(7437.5396) 0.78904
4 tensor(7870.9746) 0.81544
5 tensor(7573.1807) 0.81032
6 tensor(7508.7705) 0.80456
7 tensor(7309.7900) 0.8078
8 tensor(7347.5527) 0.8046
9 tensor(7413.9971) 0.79484
10 tensor(7382.2266) 0.7932
11 tensor(7062.0210) 0.80036
12 tensor(7007.3374) 0.79356
13 tensor(7094.6006) 0.79688
14 tensor(7276.3970) 0.79648
15 tensor(7120.0381) 0.7966
16 tensor(7182.1431) 0.79264
17 tensor(7240.6724) 0.79588
18 tensor(7072.0029) 0.79568
19 tensor(7178.2671) 0.79712
20 tensor(7055.6724) 0.79064
21 tensor(7227.4644) 0.79972
22 tensor(7165.8589) 0.79832
23 tensor(7107.2520) 0.79828
24 tensor(7048.2095) 0.79324
25 tensor(7111.5381) 0.79804
26 tensor(7059.1777) 0.79612
27 tensor(7079.7021) 0.79792
28 tensor(7057.7715) 0.79728
29 tensor(7009.0581) 0.7976
30 tensor(7098.6797) 0.7942
31 tensor(7095.1919) 0.79672
32 tensor(7046.9438) 0.79592
33 tensor(7146.7944) 0.79708
34 tensor(7081.5288) 0.79692
3

In [0]:
# Collecting test data
test_filepath = ['/test/pos/', 'test/neg/']
test_we = []

In [0]:
for directory in test_filepath:
    for index, file in enumerate(os.listdir(path + directory)):
        with open(path + directory + file, 'r') as f:
            text = f.read().strip('\n').lower()
            words = re.findall('[a-z]+', text)
            weights = []
            for word in words:
              if word_idx.get(word) and abs(idx_sent[word_idx[word]]) >= 1.0:
                weights.append(idx_sent[word_idx[word]])
              else:
                continue
            weights = torch.FloatTensor(weights) 
            weights = weights.view(1,1,-1) # (input_size, batch_size, time_steps)
            match = re.search('_[0-9]+', file)
            Y = 0.1 * int(match.group()[1:])
            Y = torch.FloatTensor([Y])
            Y = Y.view(1,1)
            test_we.append((weights, Y))

In [0]:
# Validation
total_cost = 0
acc = 0
for i,j in test_we:
  pred = sentiment.forward(i)
  acc += (pred.round() == j.round())
  # total_cost += sentiment_glove.backward(j, lr=0.001)
print(acc.item()/len(test_we)*100)

The sentiment of each word are particularly trained on IMDB dataset, so it performs well here. It is not assured to give better result on other tasks.

Notes:
    Removing stopwords may help, but some negatively corelated words such as haven't, wasn, shouldn't also eliminated(based on nltk stopwords). If we use more layers of neural networks, irrelevant words will be automatically filtered.

In [0]:
def normalize_tensor(l):
    return 0.01*(l - l.mean(dim = -1, keepdim = True))/(l.std(dim = -1, keepdim = True, unbiased = False) + 1e-6)

In [0]:
temp_a = torch.randn(2,5)*0.01

In [0]:
for i in range(50):
  print(temp_a)
  temp_a = normalize_tensor(temp_a)